In [ ]:
def get_parent_directory_path():
    return os.path.dirname(
        os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
    )

def add_directory_to_sys_path(directory_path):
    return sys.path.insert(-1, directory_path)

In [ ]:
import numpy as np
import pickle
import tensorflow as tf
from tqdm import *
import telegram_send
import os, sys, inspect
add_directory_to_sys_path(get_parent_directory_path())

from dataloader import DataLoader
import lstm as lstm_utils
import validate as validator
from constant import Constant
from parameters import Parameters

In [ ]:
## utilities.py

def get_numbers_of_batch(dataloader):
    return dataloader.num_batch()


def load_dataloader(METADATA_PATH, BATCH_SIZE, LABEL_COLUMN_NAME, EXTRACTED_FEATURE_NAME=None, is_training=False):
    return DataLoader(file_path=METADATA_PATH
                                  , batch_size=BATCH_SIZE
                                  , label_column_name=LABEL_COLUMN_NAME
                                  , is_training=is_training
                                  , use_extracted_feature=EXTRACTED_FEATURE_NAME
                                  )

def get_batch(train_dataloader):
    return train_dataloader.next_batch()


def chunk_batch_X(batch_X, chunk_size, timestep):
    return list(
        map(
            lambda X: X[:, timestep*chunk_size:(timestep+1)*chunk_size]
            , batch_X
        )
    )

def split_X(batch_X, split_indices):
    """
    params @batch_X [a list of ndarray] (length : batch_size)
    params @split_indices [a list of integer]
    
    """
    
    def debug():
        def test(X):
            return list(map(lambda X: X.shape,  np.hsplit(X, split_indices)))
        print(list(map(lambda X: test(X), batch_X)))
    # debug()
    
    return list(map(lambda X: np.hsplit(X, split_indices), batch_X))

def get_split_indices(IMAGE_WIDTH, WINDOW_NUM):
    return list(map(lambda i: i*int(IMAGE_WIDTH/WINDOW_NUM), range(1, WINDOW_NUM+1)))

def get_current_chunk(batch_X_chunks, selected_window_index):
    return list(
        map(
            lambda X_chunks: X_chunks[selected_window_index]
            , batch_X_chunks
        )
   )

def get_context_chunk(batch_X_chunks, selected_window_index, context_window_size=1):
    """
    params @batch_X_chunks[a list of a list of ndarrays]
    params @selected_window
    """
    def _index_clamp_to_positive(index): 
        return 0 if index < 0 else index
    
    def _get_before_chunk_indices():
        return _index_clamp_to_positive(selected_window_index-context_window_size), selected_window_index
    
    def _get_after_chunk_indices():
        return selected_window_index+1, selected_window_index+1+context_window_size
        
    return list(
        map(
            lambda X_chunks: X_chunks[_get_before_chunk_indices()[0]:_get_before_chunk_indices()[1]]\
                              + X_chunks[_get_after_chunk_indices()[0]:_get_after_chunk_indices()[1]]
            , batch_X_chunks
        )
   )

def initialize_batch_votings(BATCH_SIZE, N_CLASS):
    return np.zeros([BATCH_SIZE, N_CLASS])

In [7]:
### models.py

def set_X(IMAGE_HEIGHT, window_size):
    return tf.placeholder(tf.float32, [None, IMAGE_HEIGHT, window_size])

def set_window_size(IMAGE_WIDTH, WINDOW_NUM):
    return int(IMAGE_WIDTH/WINDOW_NUM)

def set_y(N_CLASS):
    return tf.placeholder(tf.float32, [None, N_CLASS])

def unstack_X_by_timestep(X, window_size):
    return tf.unstack(X, num=window_size, axis=2)

def set_rnn_cell(RNN_TYPE, EMBEDDING_DIMENSION):
    return RNN_TYPE(EMBEDDING_DIMENSION, state_is_tuple=False )

def wrap_attention(rnn_cell, attention_length):
    return tf.contrib.rnn.AttentionCellWrapper(
        rnn_cell, attention_length
        , state_is_tuple=False
    )

def stack_rnn_cells(make_rnn_cell, num_layers):
    return list(map(
        lambda i: make_rnn_cell
        , range(num_layers)
    ))

def layer_rnn_cells(stacked_rnn_cells):
    return tf.nn.rnn_cell.MultiRNNCell(
        stacked_rnn_cells
        , state_is_tuple=False
    )

def run_rnn(rnn_cells, X):
    return tf.contrib.rnn.static_rnn(
        rnn_cells, X, dtype=tf.float32
    )

def set_fully_connected_layer(X, num_hidden_units, activation):
    return tf.layers.dense(X, num_hidden_units, activation=activation)

def get_logits(X, N_CLASS):
    return tf.layers.dense(X, N_CLASS)

def calculate_losses(LOSS_FUNCTION, labels, logits):
    return LOSS_FUNCTION(labels, logits)

def calculate_mean_loss(losses):
    return tf.reduce_mean(losses)

def set_train_step(Optimizer, target_loss):
    return Optimizer.minimize(target_loss)

def build_attention_model(project_constant, data_constant, experiment_parameters):
    X = set_X(
        data_constant.IMAGE_HEIGHT
          , set_window_size(
              data_constant.IMAGE_WIDTH
              , experiment_parameters.WINDOW_NUM
          )
    )
    
    y = set_y(experiment_parameters.N_CLASS)
        
    # unstack X
    unstacked_X = unstack_X_by_timestep(
        X
        , set_window_size(
          data_constant.IMAGE_WIDTH
          , experiment_parameters.WINDOW_NUM
        )
    )
    
    # stack rnn cells
    stacked_rnn_cells = []
    for i in range(experiment_parameters.NUM_RNN_LAYERS):
        stacked_rnn_cells.append(
          wrap_attention(
                set_rnn_cell(
                    experiment_parameters.RNN_TYPE
                    , experiment_parameters.EMBEDDING_DIMENSION
                )
                , experiment_parameters.ATTENTION_LENGTH
            )
        )
    
    # run rnn
    _, encoding = run_rnn(
        layer_rnn_cells(stacked_rnn_cells)
        , unstacked_X
    )
   
    # classification
    logits = get_logits(
        set_fully_connected_layer(
            encoding
            , experiment_parameters.NUM_FC_HIDDEN_UNITS
            , experiment_parameters.Activation
        )
        , experiment_parameters.N_CLASS
    )
    
    # loss
    mean_loss = calculate_mean_loss(
        calculate_losses(experiment_parameters.LOSS_FUNCTION
                         , labels=y, logits=logits)
    )
    train_step = set_train_step(experiment_parameters.Optimizer, mean_loss)
    
    # accuracy
    prediction_indices = tf.argmax(logits, 1)
    #accuracy = tf.reduce_mean(tf.cast(predictions, "float"))
    
    return X, y, logits, mean_loss, train_step, prediction_indices

def get_batch_predictions(batch_votings, labels):
    return np.equal(np.argmax(batch_votings, 1), np.argmax(labels, 1))

def add_votings(batch_votings, prediction_indices):
    batch_votings[np.arange(prediction_indices.shape[0]), prediction_indices] += 1
    return batch_votings

def calculate_accuracy(batch_predictions):
    return np.mean(batch_predictions)

In [9]:
### MAIN.py
def main(project_constant, data_constant, experiment_parameters):
    try:
        train_dataloader = load_dataloader(project_constant.METADATA_PATH
                        , experiment_parameters.BATCH_SIZE
                        , project_constant.LABEL_COLUMN_NAME
                        , data_constant.FEATURE_NAME
                        , is_training=True
                       )

        with tf.Graph().as_default():
            X, y, logits, mean_loss, train_step, prediction_indices = build_attention_model(
                project_constant
                , data_constant
                , experiment_parameters
            )
                        
            with tf.Session() as sess:
                sess.run(tf.global_variables_initializer())
                #for epoch in range(EPOCH_NUM):
                for epoch in range(1):

                    #for i in range(get_numbers_of_batch(train_dataloader)):
                    for batch_index in range(1):
                        batch_X, batch_y = get_batch(train_dataloader)
                        batch_votings = initialize_batch_votings(
                            experiment_parameters.BATCH_SIZE
                            , experiment_parameters.N_CLASS
                        )
                        batch_X_chunks = split_X(batch_X
                                                 , get_split_indices(
                                                     data_constant.IMAGE_WIDTH
                                                     , experiment_parameters.WINDOW_NUM
                                                 )
                                                )
                        #for selected_window_index in tnrange(experiment_parameters.WINDOW_NUM):
                        for selected_window_index in tnrange(2):
                            current_chunk = get_current_chunk(batch_X_chunks, selected_window_index)
                            feed_dict = {X: current_chunk, y: batch_y}
                            mean_loss_, prediction_indices_ = sess.run([mean_loss, prediction_indices], feed_dict)
                            batch_votings = add_votings(batch_votings, prediction_indices_)

                        print(batch_votings)
                        batch_predictions = get_batch_predictions(batch_votings, labels=batch_y)
                        accuracy = calculate_accuracy(
                            get_batch_predictions(batch_votings, labels=batch_y)
                        )
                        
                        telegram_send.send(messages=[str(batch_index)
                                                    , str(batch_predictions)
                                                    , str(accuracy)]
                                          )
                        

                            #context_chunk = get_context_chunk(batch_X_chunks, selected_window_index
                            #                                  , context_window_size=1)
    except Exception as e:
        telegram_send.send(messages=[str(e)])

    return None

main(Constant.Project2, Constant.Data.ChromaStftHop512, Parameters.Attention.Experiment1)

5863



[[ 0.  0.  2.]
 [ 0.  0.  2.]
 [ 0.  0.  2.]
 [ 0.  0.  2.]
 [ 0.  0.  2.]
 [ 0.  1.  1.]
 [ 0.  1.  1.]
 [ 0.  0.  2.]
 [ 0.  0.  2.]
 [ 0.  0.  2.]
 [ 0.  0.  2.]
 [ 0.  0.  2.]
 [ 0.  0.  2.]
 [ 0.  1.  1.]
 [ 0.  0.  2.]
 [ 0.  0.  2.]
 [ 0.  0.  2.]
 [ 0.  0.  2.]
 [ 0.  0.  2.]
 [ 0.  0.  2.]
 [ 0.  0.  2.]
 [ 0.  0.  2.]
 [ 0.  0.  2.]
 [ 0.  0.  2.]
 [ 0.  0.  2.]
 [ 0.  0.  2.]
 [ 0.  0.  2.]
 [ 0.  0.  2.]
 [ 0.  0.  2.]
 [ 0.  0.  2.]]


In [ ]:
# playground.py
project_constant = Constant.Project2
data_constant = Constant.Data.ChromaStftHop512
experiment_parameters = Parameters.Attention.Experiment1

def telegram_try_except_test():
    try:
        raise ValueError("Some Errors!")
    except Exception as e:
        aaa = np.zeros([2, 3])
        bbb = 1.239
        ccc = [1, 2, 3]
        telegram_send.send(messages=[str(aaa), str(bbb), str(ccc)])
        
telegram_try_except_test()

def set_np_array():
    BATCH_SIZE = experiment_parameters.BATCH_SIZE
    N_CLASS = experiment_parameters.N_CLASS
    aaa = np.zeros([BATCH_SIZE, N_CLASS])
    bbb = np.random.rand(BATCH_SIZE, N_CLASS)
    
    aaa += bbb
    chunk = np.zeros([BATCH_SIZE, N_CLASS])
    max_indices = np.argmax(aaa, 1)
    print(max_indices)
    print(max_indices.shape[0])
    chunk[np.arange(max_indices.shape[0]), max_indices] += 1
    print(chunk)
    
    train_dataloader = load_dataloader(project_constant.METADATA_PATH
                    , experiment_parameters.BATCH_SIZE
                    , project_constant.LABEL_COLUMN_NAME
                    , data_constant.FEATURE_NAME
                    , is_training=True
                   )
    _, batch_y = get_batch(train_dataloader)
    
    predictions = np.equal(np.argmax(aaa, 1), np.argmax(batch_y, 1))
    accuracy = np.mean(predictions)
    #print(accuracy)
    
#set_np_array()

def test(project_constant, data_constant, experiment_parameters):
    train_dataloader = load_dataloader(project_constant.METADATA_PATH
                    , experiment_parameters.BATCH_SIZE
                    , project_constant.LABEL_COLUMN_NAME
                    , data_constant.FEATURE_NAME
                    , is_training=True
                   )
    _, batch_y = get_batch(train_dataloader)
    print(batch_y)
    
    return None

#test(Constant.Project2, Constant.Data.ChromaStftHop512, Parameters.Attention.Experiment1)

"""
def small_test(experiment_parameters):
    return list(
        map(
        lambda i: wrap_attention(
            set_rnn_cell(
                experiment_parameters.RNN_TYPE
                , experiment_parameters.NUM_HIDDENS
            )
            , experiment_parameters.ATTENTION_LENGTH
        )
        , [1, 2]
    )
               )
"""
        